In [58]:
import sort_RAFT_table as srt
import pandas as pd
import numpy as np
import plotly.express as px
import re
from scipy.optimize import curve_fit

In [3]:
# plotting the kinetic curves of the RAFT polymerization
# conversion over time

# get all conversion values
conversion_list = []
for column in srt.df.columns:
    if "yield" in column:
        conversion_list.append(column)
print(conversion_list)

# get the hours from the headers with regex
hours_list = []
two_digit_regex = r"\d+"
for column in conversion_list:
    hours_list.append(int(re.findall(two_digit_regex, column)[0]))

print (hours_list)

['t0h-yield', 't1h-yield', 't2h-yield', 't4h-yield', 't8h-yield', 't15h-yield']
[0, 1, 2, 4, 8, 15]


In [29]:
# plotting the kinetic curves of the RAFT polymerization
# create a dataframe to append to
kinetic_curves = []
for index, polymerisation_kinetic in srt.df.iterrows():
    kinetic_curve_entries = pd.DataFrame({"conversion" : polymerisation_kinetic[conversion_list], "time" : hours_list})
    kinetic_curve_entries["experiment"] = polymerisation_kinetic["Experiment number"]+"_"+polymerisation_kinetic["monomer"]+"_"+polymerisation_kinetic["RAFT-Agent"]

    kinetic_curves.append(kinetic_curve_entries)
    if index == 10:
        break

kinetic_curves_df = pd.concat(kinetic_curves)
    # fig = px.line(x=hours_list, y=yields, title=f"Kinetic curve of {polymerisation_kinetic['Experiment number']} polymerization")
    # fig.show()


In [30]:
px.line(kinetic_curves_df, x="time", y="conversion", color="experiment")

In [66]:
# and fit curve and the derivation
# and find the maximum of the derivation
# intersect/superimpose all curves


    # L is responsible for scaling the output range from [0,1] to [0,L]
    # b adds bias to the output and changes its range from [0,L] to [b,L+b]
    # k is responsible for scaling the input, which remains in (-inf,inf)
    # x0 is the point in the middle of the Sigmoid, i.e. the point where Sigmoid should originally output the value 1/2 [since if x=x0, we get 1/(1+exp(0)) = 1/2].
def sigmoid (x, L ,x0, k, b):
    y = L / (1 + np.exp(-k*(x-x0)))+b
    return (y)

# def sigmoid_derivative(x, L ,x0, k, b):
#     y = (L*k*np.exp(-k*(x-x0)))/((np.exp(-k*(x-x0))+1)**2)
#     return (y)

for kinetic_curve in kinetic_curves:
    ydata = kinetic_curve["conversion"].tolist()
    xdata = kinetic_curve["time"].tolist()
    p0 = [max(ydata), np.median(xdata), 1, min(ydata)] # this is an mandatory initial guess
    popt, pcov = curve_fit(sigmoid, xdata, ydata, p0, maxfev=800*10)
    print(popt)
    # plot the fit and data
    fig = px.line(x=xdata, y=ydata, title="kinetic curve fit")
    extended_xdata = np.array(list(range(min(xdata)-2, min(xdata)-1, 1)) + xdata + list(range(max(xdata)+1, max(xdata)+3, 1)))
    fig.add_scatter(x=extended_xdata, y=sigmoid(extended_xdata, *popt), mode="lines", opacity=0.5)

    fig.show()
    print(sigmoid(extended_xdata, *popt))
    break



[ 6.45329236e+02 -1.00615016e+02  6.66153070e-02 -6.44523710e+02]


[-0.09848816  0.01413731  0.06507978  0.11274655  0.19908036  0.3408344
  0.51393976  0.53272305  0.55029684]


In [50]:
type(popt)

tuple